In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# Utilities
# We define a few utilities for data conversion and visualization to make our code more neat.
def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2


def norm(x):
    return (x - train_stats['mean']) / train_stats['std']


def plot_diff(y_true, y_pred, title=''):
    plt.scatter(y_true, y_pred)
    plt.title(title)
    plt.xlabel('True Values')
    plt.ylabel('Predictions')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim(plt.xlim())
    plt.ylim(plt.ylim())
    plt.plot([-100, 100], [-100, 100])
    plt.show()


def plot_metrics(metric_name, title, ylim=5):
    plt.title(title)
    plt.ylim(0, ylim)
    plt.plot(history.history[metric_name], color='blue', label=metric_name)
    plt.plot(history.history['val_' + metric_name], color='green', label='val_' + metric_name)
    plt.show()

In [3]:
# Prepare the data
# Get the data from UCI dataset
URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00242/ENB2012_data.xlsx'
# Use pandas excel reader
df = pd.read_excel(URL)
df = df.sample(frac=1).reset_index(drop=True)

In [4]:
df

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.62,808.5,367.5,220.50,3.5,5,0.25,5,13.86,14.75
1,0.76,661.5,416.5,122.50,7.0,2,0.25,4,36.95,37.20
2,0.76,661.5,416.5,122.50,7.0,5,0.25,5,36.70,36.15
3,0.66,759.5,318.5,220.50,3.5,5,0.10,3,11.61,13.83
4,0.98,514.5,294.0,110.25,7.0,5,0.40,4,32.74,33.88
...,...,...,...,...,...,...,...,...,...,...
763,0.62,808.5,367.5,220.50,3.5,4,0.10,3,12.71,14.14
764,0.71,710.5,269.5,220.50,3.5,4,0.10,5,10.67,14.26
765,0.69,735.0,294.0,220.50,3.5,2,0.25,1,12.78,15.21
766,0.90,563.5,318.5,122.50,7.0,4,0.40,4,35.40,39.22


In [ ]:
# Split the data into train and test with 80 train / 20 test
train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()

# Get Y1 and Y2 as the 2 outputs and format them as np arrays
train_stats.pop('Y1')
train_stats.pop('Y2')
train_stats = train_stats.transpose()
train_Y = format_output(train)
test_Y = format_output(test)

# Normalize the training and test data
norm_train_X = norm(train)
norm_test_X = norm(test)

In [ ]:
# Define model layers.
# Notice that we can specify a list of outputs (i.e. [y1_output, y2_output]) when we instantiate the Model() class.
input_layer = tf.keras.layers.Input(shape=(len(train .columns),))
first_dense = tf.keras.layers.Dense(units='128', activation='relu')(input_layer)
second_dense = tf.keras.layers.Dense(units='128', activation='relu')(first_dense)

# Y1 output will be fed directly from the second dense
y1_output = tf.keras.layers.Dense(units='1', name='y1_output')(second_dense)
third_dense = tf.keras.layers.Dense(units='64', activation='relu')(second_dense)

# Y2 output will come via the third dense
y2_output = tf.keras.layers.Dense(units='1', name='y2_output')(third_dense)

# Define the model with the input layer and a list of output layers
model = tf.keras.models.Model(inputs=input_layer, outputs=[y1_output, y2_output])
print(model.summary())

# Specify the optimizer, and compile the model with loss functions for both outputs
model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.001),
              loss={'y1_output': 'mse', 'y2_output': 'mse'},
              metrics={'y1_output': tf.keras.metrics.RootMeanSquaredError(),
                       'y2_output': tf.keras.metrics.RootMeanSquaredError()})

In [ ]:
# Train the model for 2000 epochs
history = model.fit(norm_train_X, train_Y,
                    epochs=2000, batch_size=10, validation_data=(norm_test_X, test_Y))

# Test the model and print loss and mse for both outputs
loss, Y1_loss, Y2_loss, Y1_rmse, Y2_rmse = model.evaluate(x=norm_test_X, y=test_Y)
print("Loss = {}, Y1_loss = {}, Y1_mse = {}, Y2_loss = {}, Y2_mse = {}".format(loss, Y1_loss, Y1_rmse, Y2_loss, Y2_rmse))

# Plot the loss and mse
Y_pred = model.predict(norm_test_X)
plot_diff(test_Y[0], Y_pred[0], title='Y1')
plot_diff(test_Y[1], Y_pred[1], title='Y2')
plot_metrics(metric_name='y1_output_root_mean_squared_error', title='Y1 RMSE', ylim=6)
plot_metrics(metric_name='y2_output_root_mean_squared_error', title='Y2 RMSE', ylim=7)